In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

     |████████████████████████████████| 7.2 MB 666 kB/s eta 0:00:01
     |████████████████████████████████| 300 kB 666 kB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=c2be83874ca77663be36e78b2629b7e3684e4c68009e9bc1177afbe339017f13
  Stored in directory: /Users/leah/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [9]:
import pandas as pd
import sklearn
import sklearn.datasets

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [8]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition','koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad', 'koi_kepmag']]
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]
selected_features

,koi_disposition,koi_period,koi_time0bk,koi_slogg,koi_srad,koi_kepmag
0,CONFIRMED,54.418383,162.513840,4.467,0.927,15.347
1,FALSE POSITIVE,19.899140,175.850252,4.544,0.868,15.436
2,FALSE POSITIVE,1.736952,170.307565,4.564,0.791,15.597
3,CONFIRMED,2.525592,171.595550,4.438,1.046,15.509
4,CONFIRMED,4.134435,172.979370,4.486,0.972,15.714
...,...,...,...,...,...,...
6986,FALSE POSITIVE,8.589871,132.016100,4.296,1.088,14.478
6987,FALSE POSITIVE,0.527699,131.705093,4.529,0.903,14.082
6988,CANDIDATE,1.739849,133.001270,4.444,1.031,14.757
6989,FALSE POSITIVE,0.681402,132.181750,4.447,1.041,15.385


# Create a Train Test Split

Use `koi_disposition` for the y values

In [11]:
# Assign X (data) and y (target- koi_disposition)

X = selected_features[['koi_period', 'koi_time0bk', 'koi_slogg', 'koi_srad', 'koi_kepmag']]
y = selected_features["koi_disposition"].values.reshape(-1, 1)

print(X.shape, y.shape)

(6991, 5) (6991, 1)


In [12]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [33]:
X_train.head()

,koi_period,koi_time0bk,koi_slogg,koi_srad,koi_kepmag
3563,10.548413,139.064020,4.387,1.092,13.204
4099,24.754385,140.207320,4.519,0.804,15.514
5460,1.057336,131.792007,4.594,0.683,15.414
1091,201.118319,187.569860,4.447,0.954,13.328
5999,91.649983,175.715600,4.145,1.608,12.964


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [65]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

In [67]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [68]:
print(X_train_scaled)

[[0.01519053 0.02964468 0.81453115 0.00650775 0.54497969]
 [0.03616486 0.03147197 0.84185469 0.00458743 0.76842716]
 [0.00117749 0.01802212 0.85737942 0.00378063 0.75875411]
 ...
 [0.00119659 0.01809225 0.86917822 0.0035806  0.80828013]
 [0.02729454 0.04253738 0.80935624 0.00629438 0.8002515 ]
 [0.03075576 0.02149218 0.84744359 0.00468078 0.64354808]]


In [69]:
print(X_test_scaled)

[[0.09767587 0.10808061 0.85924239 0.00428738 0.81379377]
 [0.22164694 0.13547671 0.8294349  0.0055876  0.73786032]
 [0.5489816  0.45451782 0.81949907 0.00651442 0.74569549]
 ...
 [0.07998353 0.05498312 0.83647278 0.00474079 0.75672277]
 [0.03233627 0.04466793 0.72303871 0.01086181 0.5285355 ]
 [0.00674796 0.02307292 0.66777065 0.01420903 0.42793577]]


# Train the Model



In [70]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test)

/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.4949456418081251
Testing Data Score: 0.5200228832951945


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [72]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [74]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.495, total=   0.2s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.495, total=   0.2s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.495, total=   0.2s
[CV] C=1, gamma=0.001 ................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] .................... C=1, gamma=0.001, score=0.495, total=   0.2s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.495, total=   0.2s
[CV] C=1, gamma=0.001 ................................................


/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

[CV] .................... C=10, gamma=0.01, score=0.495, total=   0.2s
[CV] C=10, gamma=0.01 ................................................
[CV] .................... C=10, gamma=0.01, score=0.495, total=   0.2s
[CV] C=10, gamma=0.01 ................................................


/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] .................... C=10, gamma=0.01, score=0.495, total=   0.2s


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    4.9s finished
/Users/leah/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [75]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'gamma': 0.0001}
0.4949456418081251


# Save the Model

In [78]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib

In [80]:
filename = 'Leah_model.sav'
joblib.dump(model, Leah_model.sav)

NameError: name 'Leah_model' is not defined